#전이학습

전이학습이란 기존의 잘 알려진 데이터, 혹은 사전학습된 모델을 업뮤 효율 증대나 도메인 확장을 위해 사용하는 학습을 의미한다.


In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from tqdm.auto import trange

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##8.1 GPU 연산확인


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [ ]:
#CIFAR 10 데이터 불러오기
transform = transforms.Compose([transforms.RandomCrop(32, padding=4),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5))])

test_transform = transforms.Compose([transforms.ToTensor(),
                                     transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5))])


trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=16, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=16, shuffle=False, num_workers=2)

In [ ]:
#pytorch에서 제공하는 사전학습된 모델 불러오기
model = torchvision.models.resnet18(weights='DEFAULT')

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 151MB/s]


In [ ]:
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
#여기서, CIFAR 10진분류이나, 이 모델은 1000개 클래스의 ImageNet으로 훈련된 것이다.
#따라서 본 데이터셋에 맞게 출력층의 노드를 10개로 변경한다.
#또한 이미지넷은 CIFAR에 비해 이미지가 크기 때문에 커널 사이즈 역시 축소해준다.
#만약 이미지가 흑백이라면 첫 번째 채널의 개수 역시 1로 조정해주어야 한다.

model.conv1 = nn.Conv2d(3, 64, kernel_size=(3,3), stride=(1,1), padding=(1,1), bias=False)
model.fc = nn.Linear(512,10)
model = model.to(device)

In [ ]:
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
#손실함수와 최적화 방법 정의

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 1e-4, weight_decay=1e-2)

In [ ]:
#사전학습 모델을 이용한 학습
num_epochs = 20
ls = 2
pbar = trange(num_epochs)

for epoch in pbar:
  correct = 0
  total = 0
  running_loss = 0.0

  for data in trainloader:
    inputs, labels = data
    inputs, labels = inputs.to(device), labels.to(device)

    optimizer.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

  cost = running_loss / len(trainloader)
  acc = 100 * correct / total

  if cost < ls:
    ls = cost
    torch.save(model.state_dict(), 'model.pth')

  pbar.set_postfix({'loss' : cost, 'train_acc' : acc})

  0%|          | 0/20 [00:00<?, ?it/s]

In [ ]:
#모델 평가, 정의된 모델 구조와 같이 만들어줘야함.
model = torchvision.models.resnet18(weights = None)
model.conv1 = nn.Conv2d(3, 64, kernel_size=(3,3), stride=(1,1), padding=(1,1), bias=False)
model.fc = nn.Linear(512,10)
model = model.to(device)

model.load_state_dict(torch.load('model.pth'))

<All keys matched successfully>

In [ ]:
correct = 0
total = 0

with torch.no_grad():
  model.eval() #Normalization, Dropout 등 비활성화
  for data in testloader:
    images, labels = data[0].to(device), data[1].to(device)
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))


Accuracy of the network on the 10000 test images: 84 %


#Model Freezing

전이학습 중, 학습이 잘 된 모델을 가져와 연구에 사용할 수 있다. 데이터가 유사한 경우에는 추가적인 전체 학습 없이도 좋은 성능이 나올 수 있다. 따라서 피쳐 추출에 해당하는 합성곱 층의 변수를 업데이트 하지 않고 분류 파트에 해당하는 변수만 업데이트 할 수 있는데, 이 때 변수가 업데이트 되지 않기위해 변수를 얼린다.


In [ ]:
#AlexNet 모델 불러오기
#pretrained=True를 하면 AlexNet 구조와 사전 학습된 파라미터를 모두 불러온다.
#False로 설정하면 AlexNet 구조만 불러온다.
model = torchvision.models.resnet18(weights = 'DEFAULT')

In [ ]:
model.conv1 = nn.Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(3, 3), bias=False)
model.fc = nn.Linear(in_features=512, out_features=10)
model = model.to(device)

In [ ]:
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
#모델 프리징
#파라미터 번호 확인하기
i = 0
for name, param in model.named_parameters():
  print(i, name)
  i += 1

0 conv1.weight
1 bn1.weight
2 bn1.bias
3 layer1.0.conv1.weight
4 layer1.0.bn1.weight
5 layer1.0.bn1.bias
6 layer1.0.conv2.weight
7 layer1.0.bn2.weight
8 layer1.0.bn2.bias
9 layer1.1.conv1.weight
10 layer1.1.bn1.weight
11 layer1.1.bn1.bias
12 layer1.1.conv2.weight
13 layer1.1.bn2.weight
14 layer1.1.bn2.bias
15 layer2.0.conv1.weight
16 layer2.0.bn1.weight
17 layer2.0.bn1.bias
18 layer2.0.conv2.weight
19 layer2.0.bn2.weight
20 layer2.0.bn2.bias
21 layer2.0.downsample.0.weight
22 layer2.0.downsample.1.weight
23 layer2.0.downsample.1.bias
24 layer2.1.conv1.weight
25 layer2.1.bn1.weight
26 layer2.1.bn1.bias
27 layer2.1.conv2.weight
28 layer2.1.bn2.weight
29 layer2.1.bn2.bias
30 layer3.0.conv1.weight
31 layer3.0.bn1.weight
32 layer3.0.bn1.bias
33 layer3.0.conv2.weight
34 layer3.0.bn2.weight
35 layer3.0.bn2.bias
36 layer3.0.downsample.0.weight
37 layer3.0.downsample.1.weight
38 layer3.0.downsample.1.bias
39 layer3.1.conv1.weight
40 layer3.1.bn1.weight
41 layer3.1.bn1.bias
42 layer3.1.conv2.wei

In [ ]:
#합성곱 층은 0~9까지이다. 따라서 9번째 변수까지 역추적을 비활성화 한 후 for문 종료
frozen = range(3, 60, 3)
for i, (name, param) in enumerate(model.named_parameters()):
  if i in frozen:
    param.requires_grad = False

In [ ]:
#requires_grad 확인
print(model.layer4[1].conv2.weight.requires_grad)
print(model.fc.weight.requires_grad)

False
True


In [ ]:
#손실함수와 최적화 방법 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 1e-4, weight_decay=1e-2)

In [ ]:
model = model.to(device)


In [ ]:
num_epochs = 20
lr = 2
pbar = trange(num_epochs)

for epoch in pbar:
    model.train()
    correct = 0
    total = 0
    running_loss = 0.0

    for inputs, labels in trainloader:
        inputs = inputs.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)  # .data 쓰지 않기
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    cost = running_loss / len(trainloader)
    acc = 100 * correct / total

    if cost < ls:
        ls = cost
        torch.save(model.state_dict(), './models/cifar10_resnet_frozen.pth')

    pbar.set_postfix({'loss': f'{cost:.4f}', 'train acc': f'{acc:.2f}%'})

  0%|          | 0/20 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
model.load_state_dict(torch.load('./models/cifar10_resnet_frozen.pth'))

In [ ]:
correct = 0
total = 0
with torch.no_grad():
  model.eval()
  for data in testloader:
    images, labels = data[0].to(device), data[1].to(device)
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))